In [1]:
from fastai.vision import *
from fastai.metrics import accuracy
from fastai.basic_data import *
import pandas as pd
from torch import optim
import re
import json
import cv2
from fastai.callbacks.hooks import num_features_model
from torch.nn import L1Loss

In [2]:
import numpy as np
import torch
import pandas as pd
import random
import string

In [3]:
SZ = 360


In [4]:
class CustomDataset(Dataset):
    def __init__(self, j, aug=None):
        self.j = j
        if aug is not None: aug = get_aug(aug)
        self.aug = aug
    
    def __getitem__(self, idx):
        item = j2anno(self.j[idx])
        if self.aug: item = self.aug(**item)
        im, bbox = item['image'], np.array(item['bboxes'][0])
        im, bbox = self.normalize_im(im), self.normalize_bbox(bbox)
        
        return im.transpose(2,0,1).astype(np.float32), bbox.astype(np.float32)
    
    def __len__(self):
        return len(self.j)
    
    def normalize_im(self, ary):
        return ((ary / 255 - imagenet_stats[0]) / imagenet_stats[1])
    
    def normalize_bbox(self, bbox):
        return bbox / SZ

In [5]:
src = (ImageList.from_folder(path='fastai-data').split_by_rand_pct(0.0).label_from_folder())

In [6]:
src

LabelLists;

Train: LabelList (1440 items)
x: ImageList
Image (3, 500, 375),Image (3, 500, 375),Image (3, 334, 500),Image (3, 500, 278),Image (3, 500, 375)
y: CategoryList
class-1059,class-1059,class-1059,class-1059,class-1059
Path: fastai-data;

Valid: LabelList (0 items)
x: ImageList

y: CategoryList

Path: fastai-data;

Test: None

In [7]:
tfms = get_transforms(do_flip=True,flip_vert=False,max_rotate=10.0,max_zoom=1.1,max_lighting=0.2,max_warp=0.2,p_affine=0.75,p_lighting=0.75)


In [8]:
data = (src.transform(tfms, size=360, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=32).normalize(imagenet_stats))

In [9]:
data

ImageDataBunch;

Train: LabelList (1440 items)
x: ImageList
Image (3, 360, 360),Image (3, 360, 360),Image (3, 360, 360),Image (3, 360, 360),Image (3, 360, 360)
y: CategoryList
class-1059,class-1059,class-1059,class-1059,class-1059
Path: fastai-data;

Valid: LabelList (0 items)
x: ImageList

y: CategoryList

Path: fastai-data;

Test: None

In [10]:
class SnakeDetector(nn.Module):
    def __init__(self, arch=models.resnet18):
        super().__init__() 
        self.cnn = create_body(arch)
        self.head = create_head(num_features_model(self.cnn) * 2, 4)
        
    def forward(self, im):
        x = self.cnn(im)
        x = self.head(x)
        return x.sigmoid_()

In [11]:
learn = Learner(data, SnakeDetector(arch=models.resnet50), loss_func=L1Loss())

In [12]:
learn.split([learn.model.cnn[:6], learn.model.cnn[6:], learn.model.head])


Learner(data=ImageDataBunch;

Train: LabelList (1440 items)
x: ImageList
Image (3, 360, 360),Image (3, 360, 360),Image (3, 360, 360),Image (3, 360, 360),Image (3, 360, 360)
y: CategoryList
class-1059,class-1059,class-1059,class-1059,class-1059
Path: fastai-data;

Valid: LabelList (0 items)
x: ImageList

y: CategoryList

Path: fastai-data;

Test: None, model=SnakeDetector(
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (

In [16]:
state_dict = torch.load('fastai-data\\models\\snake-detection-model.pth')

In [17]:
learn.model.load_state_dict(state_dict['model'])


In [18]:
learn

Learner(data=ImageDataBunch;

Train: LabelList (1440 items)
x: ImageList
Image (3, 360, 360),Image (3, 360, 360),Image (3, 360, 360),Image (3, 360, 360),Image (3, 360, 360)
y: CategoryList
class-1059,class-1059,class-1059,class-1059,class-1059
Path: fastai-data;

Valid: LabelList (0 items)
x: ImageList

y: CategoryList

Path: fastai-data;

Test: None, model=SnakeDetector(
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (

In [19]:
import os

In [20]:
import os
if not os.path.exists('cropped-images'):
    os.makedirs('cropped-images')

In [35]:
src_new = (ImageList.from_folder(path='data\\images').split_by_rand_pct(0.0).label_from_folder())

In [36]:
str_name = str(src_new.items[0])

In [37]:
str_name

'data\\images\\04c26281edbd155dca879084cdf860dc.jpg'

In [39]:
i=0
for filename in src_new.items:
    try:
        im = cv2.imread(f"{filename}", cv2.IMREAD_COLOR)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        im = cv2.resize(im, (360,360), interpolation = cv2.INTER_AREA)
        im_height, im_width, _ = im.shape
        print(str(filename))
        orig_im = cv2.imread(f"{filename}", cv2.IMREAD_COLOR)
        orig_im_height, orig_im_width, _ = orig_im.shape
        to_pred = open_image(filename)
        _,_,bbox=learn.predict(to_pred)
        im_original = cv2.imread(f"{filename}", cv2.IMREAD_COLOR)
        im_original = cv2.cvtColor(im_original, cv2.COLOR_BGR2RGB)
        im_original.shape
        im_original_width = im_original.shape[1]
        im_original_height = im_original.shape[0]
        bbox_new = bbox
        bbox_new[0] = bbox_new[0]*im_original_width 
        bbox_new[2]= bbox_new[2]*im_original_width
        bbox_new[1] = bbox_new[1]*im_original_height
        bbox_new[3] = bbox_new[3]*im_original_height
        x_min, y_min, x_max, y_max = map(int, bbox_new)
        #cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
        im_original = im_original[y_min:y_max,x_min:x_max]
        im_original = cv2.cvtColor(im_original,cv2.COLOR_BGR2RGB)
        filename_str = str(filename)
        to_save = filename_str.replace('data\\images','preprocessed-images')
        print(to_save)
        cv2.imwrite(to_save,im_original)
    except:
        pass
    if i==10:
        break
    i+=1

data\images\0a15fcd7268a4b370990b32a45349f74.jpg
preprocessed-images\0a15fcd7268a4b370990b32a45349f74.jpg
data\images\0a24d1fd4f3d75b20cbf1a3f0d2de9ea.jpg
preprocessed-images\0a24d1fd4f3d75b20cbf1a3f0d2de9ea.jpg
data\images\0a28ef17ba7d1096993b08a8b3d95acf.jpg
preprocessed-images\0a28ef17ba7d1096993b08a8b3d95acf.jpg
data\images\0a47ef9125af4c341f6e9090e733a9fe.jpg
preprocessed-images\0a47ef9125af4c341f6e9090e733a9fe.jpg
data\images\0a59b66bfc06ca9dc51ec6eacc9ce849.jpg
preprocessed-images\0a59b66bfc06ca9dc51ec6eacc9ce849.jpg
